In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
from pprint import pprint

# Importing the models to be tested
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier, Pool


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from auxiliar_func import *
from plot_func import *

In [4]:
df = pd.read_csv('Census-Income-KDD.csv')
target = 'income_50k'
df_tr, df_te = train_test_split(df, test_size=0.3, random_state=42)
df_tr = df_tr[:1000]

TARGET_METRIC = 'f1_macro'
SEED = 42

prep_params_grid = {
    'scaling': [None, 'minmax', 'standard'],
    'imputation': ['mode', 'nancat'],
    'cat_age': [False, True],
    'target_freq': [0.75, 0.8, 0.85],
    'generate_dummies': [True]
}

def n_comb(grid: dict, print_=True):
    n = 1
    for k in grid.keys():
        n *= len(grid[k])
    if print_:
        print(f'Number of model combinations to be tested: {n}')
    else:
        return n

### Tuning parameters for LDA

In [14]:
mod_par_grid = {
    'solver': ['svd', 'lsqr', 'eigen'],
    'shrinkage': [None, 'auto'],
    'store_covariance': [True, False],
    'tol': [1e-4, 1e-3, 1e-2],
}

n_comb(mod_par_grid)

lda = LDA()

results = search_best_combination(lda, mod_par_grid, prep_params_grid, df_tr, target_metric=TARGET_METRIC)
results.to_csv('results_lda.csv', index=False)

Number of model combinations to be tested: 36


### Tuning parameters for logistic regression

In [5]:
mod_par_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', None],
    'C': [0.1, 1, 10],
    'class_weight': [None, 'balanced'],
    'fit_intercept': [True, False],
    'intercept_scaling': [0.1, 1, 10],
    'max_iter': [1000],
    'multi_class': ['auto'],
    'random_state': [SEED],
    'solver': ['saga']
}

n_comb(mod_par_grid)

logreg = LogisticRegression()
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    results = search_best_combination(logreg, mod_par_grid, prep_params_grid, df_tr, target_metric=TARGET_METRIC)
results.to_csv('results_log_regression.csv', index=False)

Number of model combinations to be tested: 288


### Tuning parameters for SVM

In [7]:
mod_par_grid = {
    'penalty': ['l1', 'l2'],
    'loss': ['squared_hinge','hinge'],
    'dual': [False],
    'C': [0.1, 1, 10],
    'fit_intercept': [True, False],
    'intercept_scaling': [0.1, 1],
    'class_weight': [None, 'balanced'],
    'max_iter': [1000],
    'random_state': [SEED]
}

n_comb(mod_par_grid)

svm = LinearSVC()

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    results = search_best_combination(svm, mod_par_grid, prep_params_grid, df_tr, target_metric=TARGET_METRIC, verbose=2)
results.to_csv('results_svm.csv', index=False)

Number of model combinations to be tested: 96


### Tuning parameters for random forest

In [8]:
mod_par_grid = {
    'n_estimators': [50, 75, 100],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 20, 25, 30],
    'max_features': ['sqrt', 'log2'],
    'random_state': [SEED],
    'verbose': [0],
    'warm_start': [False],
    'class_weight': [None, 'balanced', 'balanced_subsample'],
}

n_comb(mod_par_grid)

rf = RandomForestClassifier()

results = search_best_combination(rf, mod_par_grid, prep_params_grid, df_tr, target_metric=TARGET_METRIC)
results.to_csv('results_rf.csv', index=False)

Number of model combinations to be tested: 192


### Tuning parameters for catboost

In [19]:
prep_params_grid2 = {
    'scaling': [None],
    'imputation': ['mode'],
    'cat_age': [False],
    'target_freq': [0.75, 0.8, 0.85],
    'generate_dummies': [False]
}

# a first preprocess to get the categorical features
df_tr_pre = preprocessing(df_tr, imputation='mode', cat_age=False, generate_dummies=False)
X_train, y_train = df_tr_pre.drop(target, axis=1), df_tr_pre[target]
cat_features = list(X_train.select_dtypes(include=['category']).columns)

mod_par_grid = {
    'iterations': [500],
    'depth': [6, 8, 10],
    'border_count': [32, 64, 128],
    'random_seed': [SEED],
    'verbose': [0],
    'loss_function': ['Logloss'],
    'eval_metric': ['F1', 'AUC'],
    'class_weights': [[1, 1], [1, 2], [1, 3]],
    'cat_features': [cat_features],
}

n_comb(mod_par_grid)

cat_model = CatBoostClassifier()

results = search_best_combination(cat_model, mod_par_grid, prep_params_grid2, df_tr, target_metric=TARGET_METRIC, verbose=9)
results.to_csv('results_catboost.csv', index=False)

Number of model combinations to be tested: 54


In [18]:
expanded_results = expand_dicts(results)

0      F1
1     AUC
2      F1
3     AUC
4      F1
     ... 
80     F1
81     F1
82    AUC
83    AUC
84     F1
Name: eval_metric, Length: 85, dtype: object